# Data Gathering 

## Preliminaries

In [1]:
import os
import pandas as pd
from pytrends.request import TrendReq
import pytrends
from tqdm.notebook import tqdm

In [2]:
path = 'C:\\Users\\Dominique\\Documents\\Work\\ADB\\Nowcasting\\Data'
os.chdir(path)

## Scraping Google Trends data

We define a few helper functions, <code>topic_mapper()</code> and <code>scrape_trends</code> for obtaining Google Trends search data. 

In [21]:
def topic_mapper(terms, types):
    """
    Map overarching topics for given search terms as well as associated suggestions.
    """
    keywords = []
    titles = []
    trend = TrendReq()
    for term in terms:
        l = trend.suggestions(term)
        keywords.append([
            d['mid'] for d in l
            if d['type'] in types
        ])
        titles.append([
            d['title'] for d in l
            if d['type'] in types
        ])
    keywords = [key for sublist in keywords for key in sublist]
    titles = [key for sublist in titles for key in sublist]
    mapping = dict(zip(keywords, titles))
    return mapping

In [22]:
def scrape_trends(countries, mapping, time):
    """
    Scrape search volume intensity for specified countries based on topic mapping. Save results as .csv file. 
    """
    trend = TrendReq()

    for country in tqdm(countries):
        dataset = []
        for x, v in tqdm(mapping.items()):
            #     for x in tqdm(range(0,len(keywords))):
            #         kw = [keywords[x]]
            kw = [str(x)]
            trend.build_payload(kw_list=kw, timeframe=time, geo=country)
            data = trend.interest_over_time()
            if not data.empty:
                data = data.drop(labels=['isPartial'], axis='columns')
                dataset.append(data)
        
        result = pd.concat(dataset, axis=1)
        result.rename(columns=mapping, inplace=True)
        file_name = str(country) + "_search_trends.csv"
        result.to_csv(file_name)

We now define the search terms that we are interested in. The <code>topic_mapper()</code> function automatically gets Google's suggestions, which are categorized into types. For this research, we use the following types:

1. **Topic**: The overarching search term, which has a non-intuitive tag (i.e. <code>/m/0d7pp</code> is the topic for all 'credit card'-related searches).
2. **Transportation mode**: This is relevant for flights and automotives, since we are not interested in, say, scientific research about automotive or flights.
3. **Website**: This is to capture search data for travel-related websites such as Airbnb, booking.com, and others.

In [44]:
terms = [
    'Carpool',
    'Car rental',
    'Online hotel reservations',
    'Airbnb',
    'booking.com',
    'Agoda',
    'Flights',
    'Restaurant',
    'Cafe',
    'Cinema',
    'Travel',
    'Tourism',
    'Equipment'
]

types = ['Topic', 'Transportation mode', 'Website']

mapping = topic_mapper(terms, types)

Using the <code> scrape_trends()</code> function we defined above, this snippet scrapes and downloads the search trend information for the specified countries. We use only information from 2011 onwards, as the search data are more reliable from that point onwards since Google changed their data collection system. 

In [45]:
countries = ['AU', 'CN', 'ID', 'IN', 'JP', 'KR', 'MM', 'MY', 'NZ', 'PH', 'SG', 'TH', 'TW', 'VN']
time = '2018-01-01 2020-07-24'

scrape_trends(countries, mapping, time)

# Data Cleaning

## Reshaping

### PMI
Monthly IHS Markit data from CEIC

In [ ]:
iso = pd.read_excel(path + '\iso_codes.xlsx', header=0)
iso_dict = dict(zip(iso['Alpha-3 code'], iso['Alpha-2 code']))

In [ ]:
# renaming to uniform ISO 3-codes
for key, value in iso_dict.items():
    if os.path.isfile(key + ' PMI.xlsx'):
        os.rename(key + ' PMI.xlsx', value + '_pmi.xlsx')

In [ ]:
countries = ['AU', 'CN', 'ID', 'IN', 'JP', 'KR', 'MM', 'MY', 'NZ', 'PH', 'SG', 'TH', 'TW', 'VN']
df1 = pd.DataFrame(columns = ['date', 'pmi', 'country'])

for i in countries:
    file = 'pmi/' + i + '_pmi.xlsx'
    
    df = pd.read_excel(
        file,
        skiprows=range(1, 26),
        names=['date', 'pmi'],
        usecols=[0, 1],
        parse_dates=True)

    df['country'] = i

    df1 = pd.concat([df1, df])
    
df1.to_csv("pmi.csv", index=False)

### Trends
Weekly Google Trends Data

In [57]:
countries = [
    'AU', 'CN', 'ID', 'IN', 'JP', 'KR', 'MM', 'MY', 'NZ', 'PH', 'SG', 'TH',
    'TW', 'VN'
]
df1 = pd.DataFrame(columns=[
    'date', 'country', 'hotels', 'transportation', 'flights', 'tourism',
    'restaurants', 'entertainment', 'industry'
])

for i in countries:
    file = 'search_trends/' + i + '_search_trends.csv'
    df = pd.read_csv(file)

    df['country'] = i
    #     df['credit'] = df[['Credit card', 'Cashback reward program']].mean(axis=1)
    #     df['banking'] = df[['Investment banking', 'Mobile banking', 'Online banking']].mean(axis=1)
    df['hotels'] = df[[
        'Online hotel reservations', 'Airbnb', 'Booking.com', 'Agoda'
    ]].mean(axis=1)
    df['transportation'] = df[['Carpool', 'Car rental']].mean(axis=1)
    #     df['phones'] = df[['Mobile phone', 'Mobile phone accessories']].mean(axis=1)
    df['flights'] = df['Google Flights']
    df['tourism'] = df[['Travel', 'Tourism', 'Travel visa']].mean(axis=1)
    df['restaurants'] = df[['Restaurant', 'Cafe']].mean(axis=1)
    df['entertainment'] = df[['Film']]
    df['industry'] = df[['Original equipment manufacturer',
                         'Heavy equipment']].mean(axis=1)

    df = df[[
        'date', 'country', 'entertainment', 'restaurants', 'hotels',
        'transportation', 'flights', 'tourism', 'industry'
    ]]

    df1 = pd.concat([df1, df])
    df1['date'] = pd.to_datetime(df1['date'])

df1.to_csv('search_trends.csv', index=False)

### Bond yield
Short term bond yields from national sources

In [59]:
iso = pd.read_excel(path + '\iso_codes.xlsx', header=0)
iso_dict = dict(zip(iso['Country'], iso['Alpha-2 code']))

In [60]:
df = pd.read_excel('stbond/stbond.xlsx',
                   skiprows=range(2, 26),
                   parse_dates=True,
                   header=1)
df.rename(columns={
    'Region': 'date',
    'South Korea': 'Korea (the Republic of)',
    'Taiwan': 'Taiwan (Province of China)'
}, inplace=True)
df.rename(columns=iso_dict, inplace=True)
df = df.melt(id_vars='date', value_name='stbond')
df.rename(columns={'variable':'country'}, inplace=True)
df.to_csv('stbond.csv', index=False)

### P/E ratio
Price-equity ratios from national sources

In [61]:
df = pd.read_excel('pe_ratio/pe_ratio.xlsx',
                   skiprows=range(2, 26),
                   parse_dates=True,
                   header=1)

df.rename(columns={
    'Region': 'date',
    'South Korea': 'Korea (the Republic of)',
    'Taiwan': 'Taiwan (Province of China)'
}, inplace=True)

df.rename(columns=iso_dict, inplace=True)
df.drop(columns='Hong Kong SAR (China)', inplace=True)
df = df.melt(id_vars='date', value_name='pe_ratio')
df.rename(columns={'variable':'country'}, inplace=True)
df.to_csv('pe_ratio.csv', index=False)

## Merging

In [66]:
files = [x for x in os.listdir() if x.endswith('.csv')]
name_list = []

for i in files:
    name = i.split(sep=".")[0]
    print(name)
    globals()[name] = pd.read_csv(i, parse_dates=[0])
    globals()[name]['month'] = globals()[name]['date'].dt.month
    globals()[name]['year'] = globals()[name]['date'].dt.year
    name_list.append(name)

pe_ratio
pmi
search_trends
stbond


In [83]:
from functools import reduce

merged = reduce(lambda x,y: pd.merge(x,y, on=['month','year','country'], how='left'), [search_trends, pmi,  pe_ratio, stbond])

merged = pd.concat([merged.iloc[:,0:-4], merged.iloc[:,-1], merged.iloc[:,-3]], axis=1)

merged.rename(columns={'date_y': 'mdate', 'date_x': 'wdate'}, inplace=True)
merged = merged[[
    'mdate', 'wdate', 'month', 'year', 'country', 'pmi', 'entertainment', 'restaurants', 'hotels',
        'transportation', 'flights', 'tourism', 'industry','stbond', 'pe_ratio'
]]

merged.dropna(subset=['pmi', 'entertainment', 'restaurants', 'hotels',
        'transportation', 'flights', 'tourism', 'industry'], inplace=True)

merged.drop_duplicates(subset=['pmi','entertainment'], inplace=True)

In [86]:
merged.to_csv('monthly_nc.csv', index=False)
merged.to_stata('monthly_nc.dta', write_index=False)